## background 
- 32% survival
- women,children,upper class more likely survived
- not enough boats

## load train data

In [ ]:
import pandas as pd
passenger_train=pd.read_csv('train.csv')
target_col='Survived'
id_col='PassengerId'
target=passenger_train[target_col]
total_num=len(passenger_train)

## explore the data

In [ ]:
# passenger_train.head()

In [ ]:
# passenger_train.info()

### describe number features

In [ ]:
# passenger_train.describe()

### describe category features

In [ ]:
# passenger_train.describe(include=['O'])

### correlations

In [ ]:
# passenger_train.corr()

### pivot features

In [ ]:
# passenger_train[['Pclass','Survived']].groupby('Pclass').mean().sort_values('Survived',ascending=False)

In [ ]:
# passenger_train[['Sex','Survived']].groupby('Sex').mean().sort_values('Survived',ascending=False)

### visualize

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns
# g = sns.FacetGrid(passenger_train, col='Survived')
# g.map(plt.hist, 'Age', bins=20)
# plt.show()

In [ ]:
# g = sns.FacetGrid(passenger_train, col='Survived')
# g.map(plt.hist, 'Pclass', bins=20)
# plt.show()

### drop useless cols
ticket has too many duplicates,

In [ ]:
drop_cols=[id_col,target_col,'Ticket',]

## load utils

In [ ]:
import sys
sys.path.append('../')
from util import *
from label_binary import LabelBinarizerEx
from df_pipeline import DfPipeline
from binning import Binner
from title import TitleExtractor
from cabin import HasCabin
from ensemble import EnsembleStackClassifierEx

## feature engineering

In [ ]:
from sklearn.preprocessing import Imputer,StandardScaler

def featuring(df, num_cols=[], cat_cols=[], bin_cols=[]):
    num_pipelines=[(c,Pipeline([
        ('select',DataFrameSelecter([c])),
        ('fill',Imputer(strategy='median')),
#         ('scale',StandardScaler()),
    ])) for c in num_cols]

    cat_pipelines=[(c, Pipeline([
        ('select',DataFrameSelecter([c])),
        ('encode',LabelBinarizerEx([c])),
    ])) if isinstance(c,str) else c for c in cat_cols ]
    
    bin_pipelines=[(c[0], Pipeline([
        ('select',DataFrameSelecter([c[0]])),
        ('fill',Imputer(strategy='median')),
        ('bin',c[1]),
    ])) for c in bin_cols ]

    full_pipeline=DfPipeline(num_pipelines+cat_pipelines+bin_pipelines)

    prepared_df=full_pipeline.fit_transform(df)

    return prepared_df,full_pipeline

### train with raw models

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *

def trainRawModels(train_data, target):
    clfs=[#LogisticRegression(),SGDClassifier(), KNeighborsClassifier(), GaussianNB(),
          LogisticRegression(),SVC(),RandomForestClassifier(),GradientBoostingClassifier(),
        AdaBoostClassifier(), ExtraTreesClassifier(), XGBClassifier()]
    scores=[cross_val_score(clf,train_data,target,scoring='accuracy',cv=10,n_jobs=-1).mean() for clf in clfs]

    labels=[c.__class__.__name__[:3] for c in clfs]
    X=np.arange(len(clfs))
    bar(X,scores,tick_label=labels,color='rgb')
    ylim(0.5,1.0)
    show()
    print(sorted(zip(labels,scores),key=lambda x:x[1],reverse=True)[:3])
    return clfs

## fill missing ages according to class

In [ ]:
missing_ages=passenger_train[passenger_train['Age'].isnull()]
class_ages=passenger_train.groupby(['Pclass','Sex']).median()['Age']
for i in missing_ages.index:
    passenger_train.loc[i,'Age'] = class_ages[passenger_train.iloc[i]['Pclass'],passenger_train.iloc[i]['Sex']]
passenger_train['Age'].count()

## just use num features

In [ ]:
summary=passenger_train.describe()
num_cols=list(summary.columns)
for c in drop_cols:
    if c in num_cols:
        num_cols.remove(c) 

prepared_passenger_train,full_pipeline = featuring(passenger_train,num_cols)
prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

## binarize category features

In [ ]:
cat_cols=list(set(passenger_train.columns)-set(summary.columns))
for c in ['Name','Ticket','Cabin']:
    cat_cols.remove(c)

prepared_passenger_train,full_pipeline = featuring(passenger_train,num_cols,cat_cols)
prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

## discretize num features

In [ ]:
# ages=passenger_train['Age'].quantile(np.linspace(0.1,1,7))
# ages

In [ ]:
# fares=passenger_train['Fare'].quantile(np.linspace(0.1,1,5))
# fares

In [ ]:
bin_cols=[('Age',Binner([14.,35.,50.])),
          ('Fare',Binner([8.,14.,31.,66.])),
         ]
[num_cols.remove(c[0]) for c in bin_cols]

prepared_passenger_train,full_pipeline = featuring(passenger_train,num_cols,cat_cols,bin_cols)
prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

### extract new features
extract title from name

In [ ]:
# passenger_train2=passenger_train.copy()
# passenger_train2['Title'] = passenger_train2['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# pd.crosstab(passenger_train2['Title'], passenger_train2['Sex'])

In [ ]:
l=LabelBinarizerEx(['Title'])
cat_cols.append(('Name',Pipeline([
        ('select',DataFrameSelecter(['Name'])),
        ('extract',TitleExtractor()),
        ('encode',l),
    ])))

prepared_passenger_train,full_pipeline = featuring(passenger_train,num_cols,cat_cols,bin_cols)
prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

## make new features
sibsp+parch to make IsAlone

In [ ]:
# passenger_train2= passenger_train.copy()
# passenger_train2['FamilySize']=passenger_train2['SibSp']+passenger_train2['Parch']+1
# passenger_train2['IsAlone'] = (passenger_train2['FamilySize'] == 1).astype(int)
# passenger_train2.groupby('IsAlone').mean()['Survived']

In [ ]:
# passenger_train['FamilySize']=passenger_train['SibSp']+passenger_train['Parch']+1
# passenger_train['IsAlone'] = (passenger_train['FamilySize'] == 1).astype(int)

# prepared_passenger_train = featuring(passenger_train,['Pclass','IsAlone'],['Sex','Embarked'],bin_cols)
# prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

### add feature HasCabin

In [ ]:
cat_cols.append(('Cabin',Pipeline([
        ('select',DataFrameSelecter(['Cabin'])),
        ('extract',HasCabin()),
    ])))

prepared_passenger_train,full_pipeline = featuring(passenger_train,num_cols,cat_cols,bin_cols)
prepared_passenger_train.head()

In [ ]:
# clfs=trainRawModels(prepared_passenger_train,target)

### add feature class*age

In [ ]:
# prepared_passenger_train['Pclass*Age']=prepared_passenger_train['Pclass']*prepared_passenger_train['Age']
# prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

In [ ]:
# prepared_passenger_train=prepared_passenger_train.drop(['Pclass','Age'],axis=1)
# prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

In [ ]:
# prepared_passenger_train = featuring(passenger_train,num_cols,cat_cols,bin_cols)
# prepared_passenger_train.head()

In [ ]:
# trainRawModels(prepared_passenger_train,target)

In [ ]:
# prepared_passenger_train.drop('Sex_female',axis=1,inplace=True)

In [ ]:
# trainRawModels(prepared_passenger_train,target)

In [ ]:
# prepared_passenger_train_surv=prepared_passenger_train.copy()
# prepared_passenger_train_surv['Survived']=target
# prepared_passenger_train_surv.corr()

In [ ]:
# passenger_train.corr()

In [ ]:
# from sklearn.model_selection import GridSearchCV

# pg={'max_depth':[2,3,4,5,6],'learning_rate':[0.001,0.01,0.05,0.1,0.5],'n_estimators':[50,100,200,300,500]}
# g=GridSearchCV(XGBClassifier(),param_grid=pg,scoring='accuracy',cv=10,n_jobs=-1)
# g.fit(prepared_passenger_train,target)
# print(g.best_score_, g.best_estimator_)

In [ ]:
# xgb=XGBClassifier(
#     #learning_rate = 0.02,
#  n_estimators= 2000,
#  max_depth= 4,
#  min_child_weight= 2,
#  #gamma=1,
#  gamma=0.9,                        
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread= -1,
#  scale_pos_weight=1)
# scores=cross_val_score(xgb,prepared_passenger_train,target,cv=10,n_jobs=-1,scoring='accuracy')
# scores.mean()

## stacking

In [ ]:
clfs=[#LogisticRegression(),SGDClassifier(), KNeighborsClassifier(), GaussianNB(),
          LogisticRegression(),SVC(probability=True),RandomForestClassifier(),#GradientBoostingClassifier(),
#        AdaBoostClassifier(), ExtraTreesClassifier(), XGBClassifier()
]

In [ ]:
from brew.base import Ensemble
from brew.stacking import EnsembleStackClassifier,EnsembleStack
import sklearn

layer_1 = Ensemble(clfs)
layer_2 = Ensemble([sklearn.clone(clfs[0])])

stack = EnsembleStack(cv=len(clfs))

stack.add_layer(layer_1)
stack.add_layer(layer_2)

sclf = EnsembleStackClassifierEx(stack)
scores=cross_val_score(sclf,prepared_passenger_train.values,target,cv=5,n_jobs=-1)
scores.mean()

In [ ]:
target

## test it

In [ ]:
# passenger_test=pd.read_csv('test.csv')
# test_id=passenger_test[id_col]
# drop_cols.remove(target_col)
# passenger_test.drop(drop_cols,axis=1,inplace=True)
# prepared_passenger_test=full_pl.transform(passenger_test)